In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from scipy import stats

# UPDATE ALL OF THIS.

KeyboardInterrupt: 

In [ ]:
# Read in processed data.
df = pd.read_csv('summary_fall2020.csv')

In [ ]:
n = summary_fall2020.shape[0]
#y = np.array(summary_fall2020['promis_anx'])
#X = np.zeros((n, 4))
#X[:, 0] = 1
#x1 = summary_fall2020['eta0']
#X[:, 1] = (x1 - x1.mean())/x1.std()
#x2 = summary_fall2020['avg_last8_pct_correct']
#X[:, 2] = (x2 - x2.mean())/x2.std()
#X[:, 3] = X[:, 1]*X[:, 2]
#model = sm.OLS(y, X)
model = smf.ols(formula = 'eta0 ~ promis_anx + promis_ang + promis_dep + promis_pos + bite',
                data = summary_fall2020.loc[good_perf])
results = model.fit()
# Determine if the whole model (considering all of the psych measures together) is predictive.
print(results.summary())
print(n)
print(index.sum())
print(summary_fall2020[['promis_anx', 'promis_ang', 'promis_dep', 'promis_pos', 'bite', 'eta0']].loc[good_perf].corr())

In [ ]:
sns.displot(summary_fall2020_inlab['eta0'])
sns.displot(summary_fall2020_online['eta0'])

In [ ]:
print('Demographic Breakdown (In Lab) \n')

print(summary_fall2020_inlab['sex'].value_counts())
print('\n')
print(summary_fall2020_inlab['latinx'].value_counts())
print('\n')
print(summary_fall2020_inlab['race'].value_counts())

In [ ]:
print('Demographic Breakdown (Online) \n')

print(summary_fall2020_online['sex'].value_counts())
print('\n')
print(summary_fall2020_online['latinx'].value_counts())
print('\n')
print(summary_fall2020_online['race'].value_counts())

In [ ]:
print('Raw Behavioral Scores (In Lab) \n')

score_name = ['rel_irl', 'threat_benign_os', 'threat_benign_ri']
n_score = len(score_name)
mean = np.zeros(n_score)
t_val = np.zeros(n_score)
p_val = np.zeros(n_score)
for i in range(n_score):
    mean[i] = np.round(summary_fall2020_inlab[score_name[i]].mean(), 2)
    t_test = np.round(stats.ttest_1samp(summary_fall2020_inlab[score_name[i]], popmean = 0), 4)
    t_val[i] = t_test[0]
    p_val[i] = t_test[1]
behav_scores_tests = pd.DataFrame({'score_name': score_name, 'mean': mean, 't_val': t_val, 'p_val': p_val})
print(behav_scores_tests)

In [ ]:
print('Raw Behavioral Scores (Online) \n')

score_name = ['rel_irl', 'threat_benign_os', 'threat_benign_ri']
n_score = len(score_name)
mean = np.zeros(n_score)
t_val = np.zeros(n_score)
p_val = np.zeros(n_score)
for i in range(n_score):
    mean[i] = np.round(summary_fall2020_online[score_name[i]].mean(), 2)
    t_test = np.round(stats.ttest_1samp(summary_fall2020_online[score_name[i]], popmean = 0), 4)
    t_val[i] = t_test[0]
    p_val[i] = t_test[1]
behav_scores_tests = pd.DataFrame({'score_name': score_name, 'mean': mean, 't_val': t_val, 'p_val': p_val})
print(behav_scores_tests)

In [ ]:
print('Percent Correct per Stage (In Lab) \n')

stage_name = ['tutorial_0a', 'tutorial_0b', 'tutorial_0c', 'training', 'transfer']
n_stage = len(stage_name)
mean = np.zeros(n_stage)
t_val = np.zeros(n_stage)
p_val = np.zeros(n_stage)
for i in range(n_stage):
    mean[i] = np.round(summary_fall2020_inlab[stage_name[i] + '_pct_correct'].mean(), 2)
    t_test = np.round(stats.ttest_1samp(summary_fall2020_inlab[stage_name[i] + '_pct_correct'], popmean = 50), 4)
    t_val[i] = t_test[0]
    p_val[i] = t_test[1]
pct_correct_tests = pd.DataFrame({'stage_name': stage_name, 'mean': mean, 't_val': t_val, 'p_val': p_val})
print(pct_correct_tests)

In [ ]:
print('Percent Correct per Stage (Online) \n')

stage_name = ['tutorial_0a', 'tutorial_0b', 'tutorial_0c', 'training', 'transfer']
n_stage = len(stage_name)
mean = np.zeros(n_stage)
t_val = np.zeros(n_stage)
p_val = np.zeros(n_stage)
for i in range(n_stage):
    mean[i] = np.round(summary_fall2020_online[stage_name[i] + '_pct_correct'].mean(), 2)
    t_test = np.round(stats.ttest_1samp(summary_fall2020_online[stage_name[i] + '_pct_correct'], popmean = 50), 4)
    t_val[i] = t_test[0]
    p_val[i] = t_test[1]
pct_correct_tests = pd.DataFrame({'stage_name': stage_name, 'mean': mean, 't_val': t_val, 'p_val': p_val})
print(pct_correct_tests)

In [ ]:
sns.displot(summary_fall2020_online[stage_name[0] + '_pct_correct'])

In [ ]:
print(np.round(stats.ttest_1samp(df['eta0'], popmean = 1), 4))
print(np.round(stats.ttest_1samp(df['threat_benign_os'], popmean = 0), 4))
d = df['threat_benign_os'].mean()/df['threat_benign_os'].std()
print(d)